# Model Training

In [ ]:
from EduKTM import DKT
import torch
import torch.utils.data as Data
import numpy as np
import tqdm
from sklearn.metrics import roc_auc_score
import optuna

In [ ]:
NUM_QUESTIONS = 9

def get_data_loader(data_path, batch_size, shuffle=False, data_percentage=1.0):
    data = torch.FloatTensor(np.load(data_path))
    # Select only a percentage of the data
    if data_percentage < 1.0:
        total_samples = len(data)
        samples_to_keep = int(total_samples * data_percentage)
        if shuffle:
            indices = torch.randperm(total_samples)[:samples_to_keep]
            data = data[indices]
        else:
            data = data[:samples_to_keep]
    
    data_loader = Data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    return data_loader

In [ ]:
def importance_objective(trial):
    # Define hyperparameters to optimize
    hidden_size = trial.suggest_categorical('hidden_size', [5, 10, 20, 50, 100])
    num_layer = trial.suggest_int('num_layers', 1, 3)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    
    train_subset_loader = get_data_loader( '../train_data.npy', batch_size, True, 0.1)
    test_subset_loader = get_data_loader( '../test_data.npy', batch_size, False, 0.1)
    
    # Initialize and train model
    dkt_model = DKT(NUM_QUESTIONS, hidden_size, num_layer)
    dkt_model.train(train_subset_loader, test_subset_loader, epoch=10, lr=lr)
    
    # Return the AUC score to be maximized
    return dkt_model.eval(test_subset_loader)

def plot_param_importance():
    study = optuna.create_study(study_name="importances", storage="sqlite:///importancees_study.db", load_if_exists=True, direction='maximize')
        
    study.optimize(importance_objective, n_trials=10, n_jobs=4)
    
    print(f"Best parameters: {study.best_params}")
    print(f"Best AUC: {study.best_value}")

In [ ]:
plot_param_importance()

In [ ]:


def objective(trial):
    # Define hyperparameters to optimize
    hidden_size = trial.suggest_categorical('hidden_size', [5, 10, 20, 50, 100])
    num_layer = trial.suggest_int('num_layers', 1, 3)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    
    # Get data loaders
    train_loader = get_data_loader('../train_data.npy', batch_size, True)
    test_loader = get_data_loader('../test_data.npy', batch_size, False)
    
    # Initialize and train model
    dkt_model = DKT(NUM_QUESTIONS, hidden_size, num_layer)
    dkt_model.train(train_loader, test_loader, epoch=10, lr=lr)
    
    # Return the AUC score to be maximized
    return dkt_model.eval(test_loader)

def run_optuna_optimization(n_trials=30):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print("Best parameters:", study.best_params)
    print("Best AUC:", study.best_value)
    
    # Optionally, plot optimization history
    try:
        optuna.visualization.plot_optimization_history(study)
        optuna.visualization.plot_param_importances(study)
    except:
        print("Visualization requires plotly and other dependencies")
        
    return study.best_params, study.best_value


In [ ]:
best_params, best_auc = run_optuna_optimization(n_trials=20)

In [ ]:
train_loader = get_data_loader('./train_data.npy', best_params['batch_size'], True)
test_loader = get_data_loader('./test_data.npy', best_params['batch_size'], False)

dkt = DKT(NUM_QUESTIONS, HIDDEN_SIZE, NUM_LAYERS)
dkt.train(train_loader, test_loader, epoch=20)
dkt.save("dkt.params")


In [ ]:
dkt.load("dkt.params")
auc = dkt.eval(test_loader)
print("auc: %.6f" % auc)

In [ ]:
def process_raw_pred(raw_question_matrix, raw_pred, num_questions: int) -> tuple:
    questions = torch.nonzero(raw_question_matrix)[1:, 1] % num_questions
    length = questions.shape[0]
    pred = raw_pred[: length]
    pred = pred.gather(1, questions.view(-1, 1)).flatten()
    truth = torch.nonzero(raw_question_matrix)[1:, 1] // num_questions
    
    return pred, truth

def eval(model, test_data) -> float:
    model.eval()
    y_pred = torch.Tensor([])
    y_truth = torch.Tensor([])
    for batch in tqdm.tqdm(test_data, "evaluating"):
        integrated_pred = model(batch)
        batch_size = batch.shape[0]
        for student in range(batch_size):
            pred, truth = process_raw_pred(batch[student], integrated_pred[student], NUM_QUESTIONS)
            y_pred = torch.cat([y_pred, pred])
            y_truth = torch.cat([y_truth, truth])
    return [y_pred, y_truth]

In [ ]:
model = dkt.dkt_model

y_pred, y_truth = eval(model, test_loader)

print(y_pred[:10])
print(y_truth.shape)

y_pred_binary = (y_pred >= 0.5).float()
print("Binary predictions:", y_pred_binary)

print("Accuracy:", torch.sum(y_pred_binary == y_truth)/len(y_truth))
print("y_pred_binary shape", y_pred_binary.shape)
roc_auc_score(y_truth.detach().numpy(), y_pred.detach().numpy())

In [ ]:
model = dkt.dkt_model

first_batch = next(iter(test_loader))
first_batch_element = first_batch[0]
first_batch_size_1 = first_batch_element.unsqueeze(0)
model.eval()


pred = model(first_batch_size_1)

print(pred.shape)

last_pred = pred[0, -1, :]

print(pred)

# new_dataset = first_batch_element.unsqueeze(0)

# new_test_loader = Data.DataLoader(new_dataset, batch_size=1, shuffle=False)
# print(new_test_loader.dataset.shape)
# pred_y, true_y = eval(model, new_test_loader)
# print("Predictions:", pred_y)
# print("Ground truth:", true_y)
# print("ROC AUC:", roc_auc_score(true_y.detach().numpy(), pred_y.detach().numpy()))
